## XGBoost (eXtra Gradient Boost)

XGBoost  
- 트리기반의 앙상블 학습에서 가장 각광받고 있는 알고리즘 중의 하나  
- 압도적인 수치 차이는 아니지만 분류에 있어서 일반적으로 다른 머신러닝 알고리즘 보다 뛰어난 예측 성능을 나타냄  
- GBM에 기반하지만, GBM의 단점인 느린 수행 시간 및 과적합 규제 부재 등의 문제 해결  
- 특히 병렬 CPU 환경에서 병렬 학습이 가능해 기존 GBM보다 빠르게 학습 완료

---
부스팅 (Boosting)  
- boost : 밀어 올림, 강화   
- 머신러닝에서의 부스팅  
  - 약한 모델을 여러 번 순차적으로 적용해서 강한 모델로 만들어 간다는 의미  
  - 여러 개의 분류기가 순차적으로 학습을 수행하되 
  - 이전 학습기가 잘못 예측한 데이터를 다음 학습기가 학습하여 순차적으로 예측 정확도를 높이는 결과  
  - 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측할 수 있도록  
    - 다음 분류기에게는 가중치(weight)를 부여하면서  
    - 학습과 예측을 진행  
  - 예측 성능이 뛰어나 앙상블 학습 주도  

---

XGBoost 장점  
- 뛰어난 예측 성능  
    - 일반적으로 분류와 회기 영역에서 우수한 예측 성능 발휘 
    - XGBoost 또는 LightGBM을 약한 학습기로 다시 배깅이나 보팅을 하는 것이 가능하지만
        - 수행시간이 오래 걸리고 반드시 성능이 향상되는 것이 아니고, 약간 떨어질 수도 있기 때문에  
        - XGBoost는 보통 배깅을 적용하지 않음      
- GBM 대비 빠른 수행 시간    
- 과적합 규제  
    - XGBoost 자체에 과적합 규제 기능 있어서 과적합에 좀 더 강한 내구성이 있음   
- 가지치기 (pruning)  - 더 이상 긍정 이득이 없는 분할을 가지치기 해서 분할 수를 줄이게 됨
    - max_depth 파라미터로 가지치기 조정
        - 미리 설정해 놓은 max_depth까지만 split하고 pruning
        - 거꾸로 올라가면서 positive gain이 없는 노드는 삭제 
    - 내부 동작. 별도의 하이퍼 파라미터 없음 .
- 교차 검증 내장  
    - 매 반복(iteration)마다 교차 검증 수행
    - 부스팅 프로세스 중에 가장 최적의 boosting iteration 알 수 있음 
- 결손값 자체 처리
    - 결손값(missing value)를 처리할 수 있는 기능이 내장되어 있음  
    - 특별히 Null 값을 바꾸지 않아도 알아서 Null 값을 인지하여 자체적으로 처리 
    



### XGBoost 사용법 2가지

XGBoost 파이썬 구현

- XGBoost는 처음에는 C/C++로 작성되어서, 사이킷런과 호환되지 않았음
- 즉, 사이킷런의 fit(), predict() 메소드와 GridSearchCV 등을 사용할 수 없었음

방법(1) : 파이썬 래퍼 XGBoost 모듈 사용
* 파이썬에서도 XGBoost를 구동할 수 있도록 파이썬 패키지(xgboost) 제공  
* 패키지 역할 : 대부분 C/C++ 핵심 라이브러리를 호출하는 것  
* 사이킷런과 호환되지 않는 독자적인 XGBoost 전용 패키지  
* 따라서 사이킷런의 fit(), predict() 메서드 같은 사이킷런 고유 아키텍처 적용 불가  
* 다양한 유틸리티(cross_val_score, GridSearchCV, Pipeline 등)도 사용 불가  
* '파이썬 래퍼 XGBoost 모듈'로 지칭 
 
방법(2) : 사이킷런 래퍼 XGBoost 모듈사용 
* 파이썬 기반의 ML 이용자들이 사이킷런을 많이 사용하기 때문에  
사이킷런과 연동할 수 있는 래퍼 클래스(Wrapper class) 제공
* XGBClassifer와 XGBRegressor 래퍼 클래스
*  사이킷런 estimator가 학습을 위해 사용하는 fit()과 predict() 등 
*  표준 사이킷런 개발 프로세스 및 다양한 유틸리티 활용 가능
* '사이킷런 래퍼 XGBoost 모듈'로 지칭

'파이썬 래퍼 XGBoost 모듈'과 '사이킷런 래퍼 XGBoost 모듈' 차이점
- '사이킷런 래퍼 XGBoost 모듈' : 사이킷런의 Estimator와 사용법 동일
- '파이썬 래퍼 XGBoost 모듈' : 파이썬 네이티브 XGBoost 고유의 API와 하이퍼 파라미터 이용
- 크게 다르지는 않지만 몇 가지 파라미터명들이 다름  
- 파이썬 Wrapper : 사이킷런 Wrapper
    - eta : learning_rate  (학습률)
    - num_boost_rounds : n_estimators  (학습기(트리)의 개수 : 반복 수행 횟수)    


XGBoost 조기 중단 기능 (Early Stopping)  
- 지정한 수만큼의 부스팅 반복 작업이 종료되지 않더라도  
- 예측 오류가 더이상 개선되지 않으면  
- 중간에 중지해서 수행 시간 개선  
- 즉, 오류 감소 효과 없으면 멈춤  
- 학습 시간 단축. 특히 최적화 튜닝 단계에서 적절하게 사용 가능  

- 너무 반복 횟수를 단축할 경우  
- 예측 성능 최적화가 안 된 상태에서 학습이 종료될 수 있으므로 주의 필요  

조기 중단 설정을 위한 주요 파라미터  
- early_stopping_rounds : 더 이상 비용 평가 지표가 감소하지 않는 최대 반복 횟수  
- eval_set : 평가를 수행하는 별도의 검증 데이터 세트  
    - 일반적으로 검증 데이터 세트에서 반복적으로 비용 감소 성능 평가  
- eval_metric : 반복 수행 시 사용하는 비용 평가 지표  
    - 검증에 사용되는 함수를 정의
    - 기본값은 회귀의 경우 mse, 분류일 경우 error
    - eval_metric 유형
        - mse : Mean Square Error(회귀)
        - mae : Mean Absolute Error(회귀)
        - logloss : Nagative log-likelihood(이진분류)  
            - 로그 손실은 분류 문제에서 모델의 성능을 평가하는 데 사용되는 손실 함수  
            - 모델이 예측한 확률 분포와 실제 레이블의 확률 분포 간의 차이 측정
            - 이 값이 작을수록 모델의 성능이 좋다고 판단  
        - error : Binary classification error rate (0.5 threshold)(이진분류)
        - merror : Multiclass classification error rate(멀티분류)
        - mlogloss : Multiclass logloss(멀티분류)
        - auc : Area under the Curve(분류(이진/멀티)

### XGBoost 설치  

In [ ]:
# XGBoost 설치  
# !pip install xgboost

In [1]:
# XGBoost 버전 확인

import xgboost
xgboost.__version__

'2.0.2'

### XGBoost 예제 : 위스콘신 유방암 예측

 
(2) 번째 방버 사이킷런 래퍼(Wrapper) XGBoost 적용  

사용 데이터 : 위스콘신 Breast Cancer 데이터 셋  

위스콘신 유방암 데이터 세트  
- 유방암의 악성종양, 양성종양 여부를 결정하는 이진 분류 데이터 세트  
- 종양의 크기, 모양 등의 형태와 관련한 많은 피처 포함  
- load_breast_cancer() 함수를 통해 위스콘신 유방암 데이터 세트 생성  
- label이 
    - 0 : malignant(악성 종양)   
    - 1 : benign(양성:정상)

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.datasets import load_breast_cancer


# 데이터 세트 로드
dataset = load_breast_cancer()
X_features = dataset.data
y_label = dataset.target

# 데이터 확인하기 위해 데이터 프레임으로 생성
cancer_df = pd.DataFrame(data=X_features, columns=dataset.feature_names)
cancer_df['target'] = y_label
cancer_df.head(3)

# 맨 마지막에 target 추가
# 악성 0, 양성 1

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0


In [5]:
# 학습 데이터/테스트 데이터 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, 
                                                    y_label, 
                                                    test_size=0.2, 
                                                    random_state=156)
print(X_train.shape, X_test.shape)


(455, 30) (114, 30)


## 방법(2) : 사이킷런 Wrapper XGBoost   
- 사이킷런의 기본 Estimator를 그대로 상속해서 만들었기 때문 
- fit()과 predict() 만으로 학습/예측 가능

- 하이퍼 파라미터 차이  
    - 파이썬 Wrapper : 사이킷런 Wrapper  
        - eta : learning_rate (학습률)  
        - num_boost_rounds : n_estimators (학습기의 개수 : 반복 수행 횟수)  

원래는 검증 데이터 세트로 해야 되는데  
데이터 세트가 충분하지 않아서  
train 데이터에서 검증용 데이터를 추출하기에 너무 적음  
그래서 테스트 데이터 사용했음  

#### 주의사항 : 검증 데이터 세트를 따로 생성해서 검증을 진행 해야 함
- 예제 데이터의 양이 검증용 데이터를 추출하기에 너무 적어 test용 데이터를 검증용과 테스트용으로 사용
- 실제 분석에서는 이렇게 진행하면 안됨

In [9]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(n_estimators=400,
                       learning_rate=0.1,
                       eval_metric='logloss',
                       max_depth=4)

eval_data_set = [(X_test, y_test)]

xgb_clf.fit(X_train, y_train, eval_set=eval_data_set)

y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

[0]	validation_0-logloss:0.56587
[1]	validation_0-logloss:0.50685
[2]	validation_0-logloss:0.45806
[3]	validation_0-logloss:0.41728
[4]	validation_0-logloss:0.38224
[5]	validation_0-logloss:0.35297
[6]	validation_0-logloss:0.32753
[7]	validation_0-logloss:0.30241
[8]	validation_0-logloss:0.28399
[9]	validation_0-logloss:0.26857
[10]	validation_0-logloss:0.25184
[11]	validation_0-logloss:0.23988
[12]	validation_0-logloss:0.23023
[13]	validation_0-logloss:0.21807
[14]	validation_0-logloss:0.21069
[15]	validation_0-logloss:0.20017
[16]	validation_0-logloss:0.19323
[17]	validation_0-logloss:0.18215
[18]	validation_0-logloss:0.17436
[19]	validation_0-logloss:0.16711
[20]	validation_0-logloss:0.16065
[21]	validation_0-logloss:0.15406
[22]	validation_0-logloss:0.14905
[23]	validation_0-logloss:0.14366
[24]	validation_0-logloss:0.13942
[25]	validation_0-logloss:0.13299
[26]	validation_0-logloss:0.13120
[27]	validation_0-logloss:0.12790
[28]	validation_0-logloss:0.12261
[29]	validation_0-loglos

[238]	validation_0-logloss:0.08467
[239]	validation_0-logloss:0.08471
[240]	validation_0-logloss:0.08467
[241]	validation_0-logloss:0.08472
[242]	validation_0-logloss:0.08458
[243]	validation_0-logloss:0.08452
[244]	validation_0-logloss:0.08453
[245]	validation_0-logloss:0.08465
[246]	validation_0-logloss:0.08461
[247]	validation_0-logloss:0.08466
[248]	validation_0-logloss:0.08478
[249]	validation_0-logloss:0.08473
[250]	validation_0-logloss:0.08459
[251]	validation_0-logloss:0.08454
[252]	validation_0-logloss:0.08455
[253]	validation_0-logloss:0.08460
[254]	validation_0-logloss:0.08445
[255]	validation_0-logloss:0.08458
[256]	validation_0-logloss:0.08472
[257]	validation_0-logloss:0.08467
[258]	validation_0-logloss:0.08454
[259]	validation_0-logloss:0.08455
[260]	validation_0-logloss:0.08441
[261]	validation_0-logloss:0.08446
[262]	validation_0-logloss:0.08451
[263]	validation_0-logloss:0.08465
[264]	validation_0-logloss:0.08451
[265]	validation_0-logloss:0.08453
[266]	validation_0-l

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=400,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [16]:
# early_stopping_rounds=50 설정으로 400회 실행 이전에 실행 종료
xgb_clf = XGBClassifier(n_estimators=400,
                       learning_rate=0.1,
                       eval_metric='logloss',
                       max_depth=3,
                       early_stopping_rounds=100)

eval_data_set = [(X_test, y_test)]

xgb_clf.fit(X_train, y_train, eval_set=eval_data_set)

y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

[0]	validation_0-logloss:0.56554
[1]	validation_0-logloss:0.50669
[2]	validation_0-logloss:0.45868
[3]	validation_0-logloss:0.41822
[4]	validation_0-logloss:0.38103
[5]	validation_0-logloss:0.35137
[6]	validation_0-logloss:0.32588
[7]	validation_0-logloss:0.30127
[8]	validation_0-logloss:0.28197
[9]	validation_0-logloss:0.26265
[10]	validation_0-logloss:0.24821
[11]	validation_0-logloss:0.23231
[12]	validation_0-logloss:0.22079
[13]	validation_0-logloss:0.20795
[14]	validation_0-logloss:0.19764
[15]	validation_0-logloss:0.18950
[16]	validation_0-logloss:0.18052
[17]	validation_0-logloss:0.17246
[18]	validation_0-logloss:0.16512
[19]	validation_0-logloss:0.15828
[20]	validation_0-logloss:0.15436
[21]	validation_0-logloss:0.14633
[22]	validation_0-logloss:0.13936
[23]	validation_0-logloss:0.13393
[24]	validation_0-logloss:0.13015
[25]	validation_0-logloss:0.12489
[26]	validation_0-logloss:0.12206
[27]	validation_0-logloss:0.11890
[28]	validation_0-logloss:0.11637
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=100,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=400,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

### XGBoost 모델의 예측 성능 평가
- 앞에서 get_clf_eval() 함수 작성했음 (복사해서 사용)

In [15]:
# 예측 성능 평가 수행
from evaluator import get_clf_eval
get_clf_eval(y_test, y_pred, y_pred_proba)

< Confusion Matrix >
[[34  3]
 [ 1 76]]
Accuracy : 0.9649, Precision : 0.9620, Recall : 0.9870,    F1 Score : 0.9744, ROC_AUC : 0.9951


### 이전 breast_cancer 데이터 세트 예측 모델 평가 

DecisionTreeClassifier
- 예측 정확도 : 0.9473684210526315
- 하이퍼 파라미터 튜닝 끝낸 최종 정확도: 0.9415

LogisticRegression
- 정확도 : 0.9824561403508771
- 정확도: 0.9825, 정밀도: 0.9817,  재현율: 0.9907, F1 Score: 0.9862, ROC_AUC : 0.9956

앙상블
- Voting 분류기 정확도 : 0.9561
- LogisticRegression 정확도 : 0.9474
- KNeighborsClassifier 정확도 : 0.9386

XGBoost


### 학습용 / 검증용 / 테스트용 데이터 세트로 분리
- 학습용 / 테스트용 데이터 세트 분리
  - X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size=0.2)
- 학습용을 다시 학습용/검증용 데이터 세트로 분리
  - X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1)
- fit() 할 때 사용하는 데이터 세트
  - eval_data_set = [(X_tr, y_tr), (X_val, y_val)] # 검증용 데이터 
  - xgb_clf.fit(X_tr, y_tr, eval_set=eval_data_set)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size=0.2)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [18]:
xgb_clf = XGBClassifier(n_estimators=1000,
                       learning_rate=0.1,
                       eval_metric='logloss',
                       max_depth=3,
                       early_stopping_rounds=100)

xgb_clf.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])

y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

[0]	validation_0-logloss:0.54266
[1]	validation_0-logloss:0.48540
[2]	validation_0-logloss:0.43970
[3]	validation_0-logloss:0.40153
[4]	validation_0-logloss:0.36600
[5]	validation_0-logloss:0.33938
[6]	validation_0-logloss:0.31149
[7]	validation_0-logloss:0.28942
[8]	validation_0-logloss:0.26820
[9]	validation_0-logloss:0.24884
[10]	validation_0-logloss:0.23200
[11]	validation_0-logloss:0.21880
[12]	validation_0-logloss:0.20529
[13]	validation_0-logloss:0.19597
[14]	validation_0-logloss:0.18464
[15]	validation_0-logloss:0.17655
[16]	validation_0-logloss:0.16867
[17]	validation_0-logloss:0.16236
[18]	validation_0-logloss:0.15398
[19]	validation_0-logloss:0.14834
[20]	validation_0-logloss:0.14076
[21]	validation_0-logloss:0.13596
[22]	validation_0-logloss:0.13241
[23]	validation_0-logloss:0.12766
[24]	validation_0-logloss:0.12539
[25]	validation_0-logloss:0.12242
[26]	validation_0-logloss:0.12088
[27]	validation_0-logloss:0.11863
[28]	validation_0-logloss:0.11512
[29]	validation_0-loglos

[238]	validation_0-logloss:0.09032
[239]	validation_0-logloss:0.09020
[240]	validation_0-logloss:0.09018
[241]	validation_0-logloss:0.08974
[242]	validation_0-logloss:0.08969
[243]	validation_0-logloss:0.08924
[244]	validation_0-logloss:0.08924
[245]	validation_0-logloss:0.08923
[246]	validation_0-logloss:0.08968
[247]	validation_0-logloss:0.09010
[248]	validation_0-logloss:0.08968
[249]	validation_0-logloss:0.08963
[250]	validation_0-logloss:0.09007
[251]	validation_0-logloss:0.09008
[252]	validation_0-logloss:0.08997
[253]	validation_0-logloss:0.08954
[254]	validation_0-logloss:0.08949
[255]	validation_0-logloss:0.08950
[256]	validation_0-logloss:0.08994
[257]	validation_0-logloss:0.09034
[258]	validation_0-logloss:0.09026
[259]	validation_0-logloss:0.08983
[260]	validation_0-logloss:0.08984
[261]	validation_0-logloss:0.08980
[262]	validation_0-logloss:0.09022
[263]	validation_0-logloss:0.09058
[264]	validation_0-logloss:0.09016
[265]	validation_0-logloss:0.09055
[266]	validation_0-l

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=100,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=400,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [19]:
get_clf_eval(y_test, y_pred, y_pred_proba)

< Confusion Matrix >
[[38  2]
 [ 2 72]]
Accuracy : 0.9649, Precision : 0.9730, Recall : 0.9730,    F1 Score : 0.9730, ROC_AUC : 0.9926


In [20]:
# 함수화
def XGBrun(datas, params):
    X_tr, y_tr, X_val, y_val, X_test, y_test = datas
    xgb_clf = XGBClassifier(**params)
    xgb_clf.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])
    y_pred = xgb_clf.predict(X_test)
    y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]
    get_clf_eval(y_test, y_pred, y_pred_proba)

In [21]:
datas = X_tr, y_tr, X_val, y_val, X_test, y_test

In [22]:
params = {
    'n_estimators':1000,
    'learning_rate':0.1,
    'eval_metric':'logloss',
    'max_depth':3,
    'early_stopping_rounds':100
}
XGBrun(datas, params)

[0]	validation_0-logloss:0.54266
[1]	validation_0-logloss:0.48540
[2]	validation_0-logloss:0.43970
[3]	validation_0-logloss:0.40153
[4]	validation_0-logloss:0.36600
[5]	validation_0-logloss:0.33938
[6]	validation_0-logloss:0.31149
[7]	validation_0-logloss:0.28942
[8]	validation_0-logloss:0.26820
[9]	validation_0-logloss:0.24884
[10]	validation_0-logloss:0.23200
[11]	validation_0-logloss:0.21880
[12]	validation_0-logloss:0.20529
[13]	validation_0-logloss:0.19597
[14]	validation_0-logloss:0.18464
[15]	validation_0-logloss:0.17655
[16]	validation_0-logloss:0.16867
[17]	validation_0-logloss:0.16236
[18]	validation_0-logloss:0.15398
[19]	validation_0-logloss:0.14834
[20]	validation_0-logloss:0.14076
[21]	validation_0-logloss:0.13596
[22]	validation_0-logloss:0.13241
[23]	validation_0-logloss:0.12766
[24]	validation_0-logloss:0.12539
[25]	validation_0-logloss:0.12242
[26]	validation_0-logloss:0.12088
[27]	validation_0-logloss:0.11863
[28]	validation_0-logloss:0.11512
[29]	validation_0-loglos

[238]	validation_0-logloss:0.09032
[239]	validation_0-logloss:0.09020
[240]	validation_0-logloss:0.09018
[241]	validation_0-logloss:0.08974
[242]	validation_0-logloss:0.08969
[243]	validation_0-logloss:0.08924
[244]	validation_0-logloss:0.08924
[245]	validation_0-logloss:0.08923
[246]	validation_0-logloss:0.08968
[247]	validation_0-logloss:0.09010
[248]	validation_0-logloss:0.08968
[249]	validation_0-logloss:0.08963
[250]	validation_0-logloss:0.09007
[251]	validation_0-logloss:0.09008
[252]	validation_0-logloss:0.08997
[253]	validation_0-logloss:0.08954
[254]	validation_0-logloss:0.08949
[255]	validation_0-logloss:0.08950
[256]	validation_0-logloss:0.08994
[257]	validation_0-logloss:0.09034
[258]	validation_0-logloss:0.09026
[259]	validation_0-logloss:0.08983
[260]	validation_0-logloss:0.08984
[261]	validation_0-logloss:0.08980
[262]	validation_0-logloss:0.09022
[263]	validation_0-logloss:0.09058
[264]	validation_0-logloss:0.09016
[265]	validation_0-logloss:0.09055
[266]	validation_0-l

In [23]:
params = {
    'n_estimators':1000,
    'learning_rate':0.1,
    'eval_metric':'logloss',
    'max_depth':3,
    'early_stopping_rounds':200
}
XGBrun(datas, params)

[0]	validation_0-logloss:0.54266
[1]	validation_0-logloss:0.48540
[2]	validation_0-logloss:0.43970
[3]	validation_0-logloss:0.40153
[4]	validation_0-logloss:0.36600
[5]	validation_0-logloss:0.33938
[6]	validation_0-logloss:0.31149
[7]	validation_0-logloss:0.28942
[8]	validation_0-logloss:0.26820
[9]	validation_0-logloss:0.24884
[10]	validation_0-logloss:0.23200
[11]	validation_0-logloss:0.21880
[12]	validation_0-logloss:0.20529
[13]	validation_0-logloss:0.19597
[14]	validation_0-logloss:0.18464
[15]	validation_0-logloss:0.17655
[16]	validation_0-logloss:0.16867
[17]	validation_0-logloss:0.16236
[18]	validation_0-logloss:0.15398
[19]	validation_0-logloss:0.14834
[20]	validation_0-logloss:0.14076
[21]	validation_0-logloss:0.13596
[22]	validation_0-logloss:0.13241
[23]	validation_0-logloss:0.12766
[24]	validation_0-logloss:0.12539
[25]	validation_0-logloss:0.12242
[26]	validation_0-logloss:0.12088
[27]	validation_0-logloss:0.11863
[28]	validation_0-logloss:0.11512
[29]	validation_0-loglos

[238]	validation_0-logloss:0.09032
[239]	validation_0-logloss:0.09020
[240]	validation_0-logloss:0.09018
[241]	validation_0-logloss:0.08974
[242]	validation_0-logloss:0.08969
[243]	validation_0-logloss:0.08924
[244]	validation_0-logloss:0.08924
[245]	validation_0-logloss:0.08923
[246]	validation_0-logloss:0.08968
[247]	validation_0-logloss:0.09010
[248]	validation_0-logloss:0.08968
[249]	validation_0-logloss:0.08963
[250]	validation_0-logloss:0.09007
[251]	validation_0-logloss:0.09008
[252]	validation_0-logloss:0.08997
[253]	validation_0-logloss:0.08954
[254]	validation_0-logloss:0.08949
[255]	validation_0-logloss:0.08950
[256]	validation_0-logloss:0.08994
[257]	validation_0-logloss:0.09034
[258]	validation_0-logloss:0.09026
[259]	validation_0-logloss:0.08983
[260]	validation_0-logloss:0.08984
[261]	validation_0-logloss:0.08980
[262]	validation_0-logloss:0.09022
[263]	validation_0-logloss:0.09058
[264]	validation_0-logloss:0.09016
[265]	validation_0-logloss:0.09055
[266]	validation_0-l